In [1]:
from Models import *
from Utils import *
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [5]:
# load pre-trained trigram
tokeniser = L3wTransformer()
tokeniser = tokeniser.load("/work/data/trigram/%s" % "2M_50k_trigram")

In [168]:
import sentencepiece as spm
s = spm.SentencePieceProcessor()
s.Load('/work/data/bpe/en.wiki.bpe.op50000.model')
s.EncodeAsIds("I love you ddddd 9")

[49881, 0, 2095, 1972, 36, 6450, 6450, 49881, 0]

In [211]:
import sentencepiece as spm
sp = spm.SentencePieceProcessor()
sp.Load('/work/data/bpe/en.wiki.bpe.op50000.model')
text = "test this one"

In [227]:
q = []
for text in df_july.q.tolist():
    q.append([model.index2word.index(t) if t in model.index2word else model.index2word.index('<unk>') for t in sp.EncodeAsPieces(text)])
q = pad_sequences(q, 10)

In [228]:
d = []
for text in df_july.d.tolist():
    d.append([model.index2word.index(t) if t in model.index2word else model.index2word.index('<unk>') for t in sp.EncodeAsPieces(text)])

d = pad_sequences(d, 10)

In [231]:
from keras.layers import Dense, Input, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Sequential

w2v = Sequential()
w2v.add(model.get_keras_embedding(train_embeddings=False))
w2v.add(GlobalAveragePooling1D())

pred = get_cosine_sim(w2v.predict(q), w2v.predict(d))


In [97]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format("/work/data/bpe/en.wiki.bpe.op50000.d200.w2v.bin", binary=True)

In [10]:
cosine = CosineSim(feature_num=nb_words)

In [13]:
pred = cosine.model.predict([q_july, d_july]).flatten()
auc(qrel_july, pred)

0.5060819675776647

In [3]:

pred = cosine.model.predict([q_test, d_test])
pred = convert_2_trec(df_test.q.tolist(), df_test.d.tolist(), pred, isQrel=False)
evaluate(qrel, pred)

NDCG: 0.795042
MAP: 0.690643


In [36]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

MAX_NB_WORDS = 100000
MAX_SEQUENCE_LENGTH = 30

# tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
# tokenizer.fit_on_texts(texts)
# sequences = tokenizer.texts_to_sequences(texts)

def get_pretrain_w2v(tokeniser, nb_words):
    
    with open(r"/data/t-mipha/data/agi_encoder/v4/universal/embedding_dict.pkl", "rb") as input_file:
        embeddings_index = pickle.load(input_file)
    
    
    emb_dim = len(embeddings_index['test'])
    
    word_index = tokeniser.word_index
    print('Found %s unique tokens.' % len(word_index))
    nb_words = min(nb_words, len(word_index) + 1)
    
    embedding_matrix = np.zeros((nb_words, emb_dim))
    for word, i in word_index.items():
        if i >= nb_words:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

    return embedding_matrix, nb_words, emb_dim


In [37]:
embedding_matrix, nb_words, emb_dim = get_pretrain_w2v(tokeniser, nb_words)

Found 382057 unique tokens.


In [206]:
model.get_keras_embedding?

In [42]:
from keras.layers import Dense, Input, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Sequential


# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(nb_words,
                            emb_dim,
                            weights=[embedding_matrix],
                            input_length=max_len,
                            trainable=False)

w2v = Sequential()
w2v.add(embedding_layer)
w2v.add(GlobalAveragePooling1D())

pred = get_cosine_sim(w2v.predict(q_test), w2v.predict(d_test))
pred = convert_2_trec(df_test.q.tolist(), df_test.d.tolist(), pred, False)
evaluate(qrel, pred)

NDCG: 0.517653
MAP: 0.500053


In [ ]:
class VAE_DSSM():

    def __init__(self, hidden_dim=300, latent_dim=128, nb_words=50005, activation="relu", emb=None):

        self.hidden_dim = hidden_dim
        self.latent_dim = latent_dim
        self.nb_words = nb_words
        self.activation = activation


        x = Input(shape=(self.nb_words,))
        
        emb = emb
        
        lstm = LSTM(self.hidden_dim)

        emb_x = lstm(emb(x))

        vae_loss, encoded = self.build_encoder(emb_x)
        self.encoder = Model(x, encoded)

        encoded_input = Input(shape=(self.latent_dim,))

        decoded = self.build_decoder(encoded_input)
        self.decoder = Model(encoded_input, decoded)

        self.model = Model(x, self.build_decoder(encoded))

        self.model.compile(optimizer='Adam',
                                 loss=vae_loss)
        
    def build_encoder(self, z):


        def sampling(args):
            z_mean_, z_log_var_ = args
            batch_size = K.shape(z_mean_)[0]
            latent_dim = K.shape(z_mean_)[1]
            epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0., stddev=1)
            return z_mean_ + K.exp(z_log_var_ / 2) * epsilon

        z_mean = Dense(self.latent_dim, name='z_mean', activation='linear')(z)
        z_log_var = Dense(self.latent_dim, name='z_log_var', activation='linear')(z)

        def vae_loss(x, x_decoded_mean):
            x = K.flatten(x)
            x_decoded_mean = K.flatten(x_decoded_mean)
            xent_loss = objectives.binary_crossentropy(x, x_decoded_mean)
            kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
            return xent_loss + kl_loss

        return (vae_loss, Lambda(sampling, output_shape=(self.latent_dim,), name='lambda')([z_mean, z_log_var]))
    
    def build_decoder(self, encoded):

        h = Dense(self.hidden_dim, activation=self.activation)(encoded)
        decoded = Dense(self.nb_words, activation='sigmoid', name='decoded_mean')(h)
        
        return decoded

    def get_name(self):
        return "vae_dssm_h%d_l%d_w%d_%s" % (self.hidden_dim, self.latent_dim, self.nb_words, self.activation)


    def batch_generator(self, reader, train_data, tokeniser, batch_size, max_len, nb_words):
        while True:
            for df in reader:
                q = df.q.tolist()
                if train_data == "1M_EN_QQ_log":
                    d = [i.split("<sep>")[0] for i in df.d.tolist()]
                else:
                    d = df.d.tolist()
                
                q = pad_sequences(tokeniser.texts_to_sequences(q), maxlen=max_len)
                
                q_one_hot = np.zeros((batch_size, nb_words))
                for i in range(len(q)):
                    q_one_hot[i][q[i]] = 1
                    
                
                yield q_one_hot, q_one_hot